# Testing the Mediapipe handsign recognition

In [ ]:
import mediapipe as mp
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time as t
from PIL import Image
from sklearn.model_selection import train_test_split
import csv
from IPython.display import clear_output

In [ ]:
model_path = r"C:\Users\Luis\OneDrive\Desktop\Maturaarbeit\codingMA\Mediapipe\models"
model = tf.keras.models.load_model(model_path)

In [ ]:
signs = ["right", "left", "up", "down", "fist", "hand", "V"]
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def test(frame):
    results = hands.process(frame)
        
    if results.multi_hand_landmarks: 
        points = []
        for handLandmarks in results.multi_hand_landmarks:
            for landmark in handLandmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                points.append(x)
                points.append(y)
                points.append(z)
            drawingModule.draw_landmarks(frame, handLandmarks, mp_hands.HAND_CONNECTIONS)
        
        return np.array(points)
       
    else: 
        return np.array([0,0])


In [ ]:
vc = cv2.VideoCapture(0)
cv2.namedWindow("preview")

while True:
    rval, frame = vc.read()
    res = test(frame)
    if sum(res) != 0:
        res = np.reshape(res, (1, res.shape[0]))
        res = model(res)
        cv2.putText(frame, signs[np.argmax(res)] ,(250,80), 4,2, (0,0,0),2)
        cv2.imshow('preview', frame)
        cv2.waitKey(100)
 
    elif cv2.waitKey(100) & 0xFF == ord(" "): 
        break

    else:
        cv2.imshow("preview", frame)

cv2.destroyAllWindows()
del(vc)